# Statistical Scorers: ZScore, IQR, and RobustZScore

This notebook demonstrates the three statistical anomaly scorers available in Anomsmith:
1. **ZScoreScorer**: Standard z-score based on mean and standard deviation
2. **IQRScorer**: Interquartile Range based outlier detection
3. **RobustZScoreScorer**: Robust z-score using median and MAD (Median Absolute Deviation)

Each scorer has different characteristics and is suited for different types of data distributions.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from plotsmith import plot_timeseries

from anomsmith import detect_anomalies, ThresholdRule
from anomsmith.primitives.scorers.statistical import ZScoreScorer, IQRScorer
from anomsmith.primitives.scorers.robust_zscore import RobustZScoreScorer

np.random.seed(42)


## Creating Test Data

We'll create two types of data:
1. **Normal data**: Clean Gaussian data with some outliers
2. **Contaminated data**: Data with outliers that might affect mean/std calculations


In [ ]:
def create_test_data(n: int = 200, contamination: float = 0.05, seed: int = 42) -> pd.Series:
    """Create test data with known anomalies.
    
    Args:
        n: Length of series
        contamination: Proportion of anomalies
        seed: Random seed
    """
    np.random.seed(seed)
    # Base normal data
    y = np.random.randn(n) * 2 + 10
    
    # Inject anomalies
    n_anomalies = int(n * contamination)
    anomaly_indices = np.random.choice(n, n_anomalies, replace=False)
    y[anomaly_indices] += np.random.choice([-1, 1], n_anomalies) * np.random.uniform(5, 10, n_anomalies)
    
    index = pd.date_range("2020-01-01", periods=n, freq="D")
    return pd.Series(y, index=index), anomaly_indices

# Create test data
y, true_anomaly_indices = create_test_data(n=200, contamination=0.08)
print(f"Created time series with {len(y)} points")
print(f"True anomalies: {len(true_anomaly_indices)}")
print(f"\nData statistics:")
print(y.describe())


In [ ]:
# Visualize the data
fig, ax = plot_timeseries(
    y,
    title='Test Data with Known Anomalies',
    xlabel='Date',
    ylabel='Value'
)
ax.scatter(y.index[true_anomaly_indices], y.values[true_anomaly_indices], 
          color='red', s=100, marker='x', linewidths=2, 
          label=f'True Anomalies ({len(true_anomaly_indices)})', zorder=5)
ax.legend()
plt.show()


## Comparing the Three Scorers

Let's initialize all three scorers and compare their performance.


In [ ]:
# Initialize all scorers
zscore_scorer = ZScoreScorer()
iqr_scorer = IQRScorer()
robust_zscore_scorer = RobustZScoreScorer(epsilon=1e-8)

# Fit all scorers
zscore_scorer.fit(y.values)
iqr_scorer.fit(y.values)
robust_zscore_scorer.fit(y.values)

# Define threshold rule
threshold_rule = ThresholdRule(method="quantile", value=0.95, quantile=0.95)

# Detect anomalies with each scorer
results_zscore = detect_anomalies(y, zscore_scorer, threshold_rule)
results_iqr = detect_anomalies(y, iqr_scorer, threshold_rule)
results_robust = detect_anomalies(y, robust_zscore_scorer, threshold_rule)

# Compare results
comparison = pd.DataFrame({
    'ZScore': [
        results_zscore['flag'].sum(),
        results_zscore['flag'].mean(),
        results_zscore['score'].mean(),
        results_zscore['score'].std()
    ],
    'IQR': [
        results_iqr['flag'].sum(),
        results_iqr['flag'].mean(),
        results_iqr['score'].mean(),
        results_iqr['score'].std()
    ],
    'RobustZScore': [
        results_robust['flag'].sum(),
        results_robust['flag'].mean(),
        results_robust['score'].mean(),
        results_robust['score'].std()
    ]
}, index=['Anomalies Detected', 'Anomaly Rate', 'Mean Score', 'Std Score'])

print("Scorer Comparison:")
print(comparison.round(4))


In [ ]:
# Visualize scores from all three scorers
scorers = [
    ('ZScore', results_zscore, 'blue'),
    ('IQR', results_iqr, 'green'),
    ('RobustZScore', results_robust, 'orange')
]

for name, result, color in scorers:
    anomaly_mask = result['flag'] == 1
    fig, ax = plot_timeseries(
        pd.Series(result['score'], index=y.index),
        title=f'{name} Scorer - {anomaly_mask.sum()} anomalies detected',
        xlabel='Date',
        ylabel='Score'
    )
    threshold_value = np.quantile(result['score'], 0.95)
    ax.axhline(threshold_value, color='r', linestyle='--', linewidth=2, 
              label=f'Threshold ({threshold_value:.2f})')
    ax.scatter(y.index[anomaly_mask], result['score'][anomaly_mask], 
              color='red', s=50, marker='x', linewidths=1.5, zorder=5)
    ax.legend()
    plt.show()


## Side-by-Side Detection Comparison

Let's visualize the actual detections on the time series.


In [ ]:
for name, result, color in scorers:
    anomaly_mask = result['flag'] == 1
    fig, ax = plot_timeseries(
        y,
        title=f'{name} Scorer Detection Results',
        xlabel='Date',
        ylabel='Value'
    )
    # True anomalies
    ax.scatter(y.index[true_anomaly_indices], y.values[true_anomaly_indices], 
              color='gray', s=80, marker='o', alpha=0.5, 
              label='True Anomalies', zorder=3)
    # Detected anomalies
    ax.scatter(y.index[anomaly_mask], y.values[anomaly_mask], 
              color='red', s=100, marker='x', linewidths=2, 
              label=f'Detected ({anomaly_mask.sum()})', zorder=5)
    ax.legend()
    plt.show()


## Understanding the Differences

### ZScoreScorer
- Uses mean and standard deviation
- Sensitive to outliers (outliers affect mean/std calculation)
- Best for: Clean, normally distributed data

### IQRScorer
- Uses quartiles (Q1, Q3) and Interquartile Range
- Robust to outliers
- Best for: Data with outliers, non-normal distributions

### RobustZScoreScorer
- Uses median and MAD (Median Absolute Deviation)
- Most robust to outliers
- Best for: Contaminated data, when you want outlier-robust statistics


In [ ]:
# Demonstrate robustness by comparing statistics
print("Statistical Properties Comparison:")
print("=" * 60)

stats_comparison = pd.DataFrame({
    'ZScore (mean/std)': [
        np.mean(y.values),
        np.std(y.values),
        np.mean(y.values) - 2 * np.std(y.values),
        np.mean(y.values) + 2 * np.std(y.values)
    ],
    'IQR (Q1/Q3)': [
        np.median(y.values),
        np.percentile(y.values, 75) - np.percentile(y.values, 25),
        np.percentile(y.values, 25),
        np.percentile(y.values, 75)
    ],
    'RobustZScore (median/MAD)': [
        np.median(y.values),
        np.median(np.abs(y.values - np.median(y.values))),
        np.median(y.values) - 2 * np.median(np.abs(y.values - np.median(y.values))),
        np.median(y.values) + 2 * np.median(np.abs(y.values - np.median(y.values)))
    ]
}, index=['Center', 'Spread', 'Lower Bound (-2σ)', 'Upper Bound (+2σ)'])

print(stats_comparison.round(4))


## Summary

In this notebook, we've explored:
1. **ZScoreScorer**: Standard z-score using mean and std
2. **IQRScorer**: Interquartile Range based detection
3. **RobustZScoreScorer**: Robust z-score using median and MAD

Key takeaways:
- Different scorers are suited for different data characteristics
- Robust methods (IQR, RobustZScore) are less affected by outliers
- The choice of scorer depends on your data distribution and contamination level
